# Aliens!

## Purpose
State the purpose of the notebook.

## Methodology
Quickly describe assumptions and processing steps.

## Results
Describe and comment the most important results.

## Suggested next steps
State suggested next steps, based on results obtained in this notebook.

# Setup

## Installations needed

In [1]:
#!pip install splinter
#!pip install citipy
#!pip install descartes

# Had to do a conda install geopandas in Anaconda as admin
#!pip install geopandas

## Library import
We import all the required Python libraries

In [38]:
import os
import pandas as pd
import numpy as np
import requests
import json
import time
import pandas as pd
import re
import matplotlib.pyplot as plt
import plotly.graph_objects as go

from bs4 import BeautifulSoup as bs
from splinter import Browser
from sqlalchemy import create_engine
from citipy import citipy
from matplotlib import pyplot

# Google API Key
from config import gkey

## Local library import
We import all the required local libraries libraries

# Data processing

## Set up Chromedriver path

In [39]:
# set executable path and initialize Chrome Browser
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

## Begin scrape code

In [40]:
# visit 
ufo_url = 'http://www.nuforc.org/webreports/ndxevent.html'
browser.visit(ufo_url)

# add in a pause to allow site to fully load before creating html object
time.sleep(3)

# create html object
html = browser.html

# create a beautiful soup object
soup = bs(html, 'html.parser')

# create list to house the table urls
table_urls = []
base_url = 'http://www.nuforc.org/webreports/'
    
# retrieve all urls
links = soup.find_all('a')[1:13]

# for loop to loop through items and append to list
for link in links:
     part_url = link['href']
     full_url = base_url + part_url   
     table_urls.append(full_url)
        

print(table_urls)

['http://www.nuforc.org/webreports/ndxe202002.html', 'http://www.nuforc.org/webreports/ndxe202001.html', 'http://www.nuforc.org/webreports/ndxe201912.html', 'http://www.nuforc.org/webreports/ndxe201911.html', 'http://www.nuforc.org/webreports/ndxe201910.html', 'http://www.nuforc.org/webreports/ndxe201909.html', 'http://www.nuforc.org/webreports/ndxe201908.html', 'http://www.nuforc.org/webreports/ndxe201907.html', 'http://www.nuforc.org/webreports/ndxe201906.html', 'http://www.nuforc.org/webreports/ndxe201905.html', 'http://www.nuforc.org/webreports/ndxe201904.html', 'http://www.nuforc.org/webreports/ndxe201903.html']


In [41]:
# create empty list
table_list = []

# loop through list to scrape table data and extend, rather than append to empty list
for table in table_urls:
    urls = pd.read_html(table)
    table_list.extend(urls)
    

print(table_list)

[       Date / Time                          City State      Shape  \
0    2/24/20 12:45                       Chester    NH       Disk   
1    2/24/20 06:00                       Medford    OR  Formation   
2    2/24/20 05:45                       Medford    OR      Other   
3    2/24/20 05:30                   Charlestown    RI      Light   
4    2/24/20 05:21                        Edison    NJ      Light   
..             ...                           ...   ...        ...   
391   2/1/20 11:45                   Santa Paula    CA      Other   
392   2/1/20 07:00                     Southside    AL      Light   
393   2/1/20 06:38                      Woodbine    GA        NaN   
394   2/1/20 01:34                         Aledo    TX   Triangle   
395   2/1/20 00:30  Progrresso (Yucatan)(Mexico)   NaN      Light   

        Duration                                            Summary   Posted  
0      ~1 minute  12:45 PM-shiny, disk/saucer shape seen in dayl...  2/25/20  
1      5 min

In [42]:
# concatenate all table data into one dataframe
pd.concat(table_list)

,Date / Time,City,State,Shape,Duration,Summary,Posted
0,2/24/20 12:45,Chester,NH,Disk,~1 minute,"12:45 PM-shiny, disk/saucer shape seen in dayl...",2/25/20
1,2/24/20 06:00,Medford,OR,Formation,5 minutes,"20 lights in formation across sky. ((""Starlink...",2/25/20
2,2/24/20 05:45,Medford,OR,Other,10 minutes,AT 545 AM I LOOKED UP AND SAID WHAT THE HELL I...,2/25/20
3,2/24/20 05:30,Charlestown,RI,Light,2-3 minutes,light moving at a high rate of speed,2/25/20
4,2/24/20 05:21,Edison,NJ,Light,25 seconds,DURING ISS PASS VIEWING SMALL OBJECT APPEARS T...,2/25/20
...,...,...,...,...,...,...,...
320,3/1/19 19:12,East Wenatchee,WA,Light,10 seconds,Orange pulsing light.,3/14/19
321,3/1/19 18:07,Mebane,NC,Changing,Months (since December) l,Interdimensional Beings Caught using thermal v...,3/14/19
322,3/1/19 13:28,Eagle River,AK,Cylinder,5,Driving down eagle river road looked into sky ...,3/14/19
323,3/1/19 13:00,Greenville,SC,NaN,10 minutes,Reptilian or Grey alien contact.,8/23/19


In [43]:
pd.concat(table_list).to_csv("Resources/nuforc_reports.csv", index = False)

## Begin cleaning

In [44]:
csv_file = "Resources/nuforc_reports.csv"
df = pd.read_csv(csv_file)
df.head()

,Date / Time,City,State,Shape,Duration,Summary,Posted
0,2/24/20 12:45,Chester,NH,Disk,~1 minute,"12:45 PM-shiny, disk/saucer shape seen in dayl...",2/25/20
1,2/24/20 06:00,Medford,OR,Formation,5 minutes,"20 lights in formation across sky. ((""Starlink...",2/25/20
2,2/24/20 05:45,Medford,OR,Other,10 minutes,AT 545 AM I LOOKED UP AND SAID WHAT THE HELL I...,2/25/20
3,2/24/20 05:30,Charlestown,RI,Light,2-3 minutes,light moving at a high rate of speed,2/25/20
4,2/24/20 05:21,Edison,NJ,Light,25 seconds,DURING ISS PASS VIEWING SMALL OBJECT APPEARS T...,2/25/20


In [45]:
# Lets take a look at what we are starting with
for col in df.columns: 
    print(col) 

Date / Time
City
State
Shape
Duration
Summary
Posted


In [46]:
df2 = df.loc[:, ~df.columns.str.contains('^Unnamed')]
print(df2.count())
df2.head()


Date / Time    6587
City           6557
State          6366
Shape          5835
Duration       5804
Summary        6570
Posted         6587
dtype: int64


,Date / Time,City,State,Shape,Duration,Summary,Posted
0,2/24/20 12:45,Chester,NH,Disk,~1 minute,"12:45 PM-shiny, disk/saucer shape seen in dayl...",2/25/20
1,2/24/20 06:00,Medford,OR,Formation,5 minutes,"20 lights in formation across sky. ((""Starlink...",2/25/20
2,2/24/20 05:45,Medford,OR,Other,10 minutes,AT 545 AM I LOOKED UP AND SAID WHAT THE HELL I...,2/25/20
3,2/24/20 05:30,Charlestown,RI,Light,2-3 minutes,light moving at a high rate of speed,2/25/20
4,2/24/20 05:21,Edison,NJ,Light,25 seconds,DURING ISS PASS VIEWING SMALL OBJECT APPEARS T...,2/25/20


In [47]:
#remove duplicates
df3 = df2.copy()

df3.drop_duplicates(subset=None, keep='first', inplace=True)
print(df3.count())
df3.head()

Date / Time    6557
City           6527
State          6337
Shape          5815
Duration       5784
Summary        6540
Posted         6557
dtype: int64


,Date / Time,City,State,Shape,Duration,Summary,Posted
0,2/24/20 12:45,Chester,NH,Disk,~1 minute,"12:45 PM-shiny, disk/saucer shape seen in dayl...",2/25/20
1,2/24/20 06:00,Medford,OR,Formation,5 minutes,"20 lights in formation across sky. ((""Starlink...",2/25/20
2,2/24/20 05:45,Medford,OR,Other,10 minutes,AT 545 AM I LOOKED UP AND SAID WHAT THE HELL I...,2/25/20
3,2/24/20 05:30,Charlestown,RI,Light,2-3 minutes,light moving at a high rate of speed,2/25/20
4,2/24/20 05:21,Edison,NJ,Light,25 seconds,DURING ISS PASS VIEWING SMALL OBJECT APPEARS T...,2/25/20


In [48]:
#only US States
aliens_df = df3[\
(df3['State']=='AL')|(df3['State']=='AK')|(df3['State']=='AZ')\
|(df3['State']=='AR')|(df3['State']=='CA')|(df3['State']=='CO')\
|(df3['State']=='CT')|(df3['State']=='DE')|(df3['State']=='FL')\
|(df3['State']=='GA')|(df3['State']=='HI')|(df3['State']=='ID')\
|(df3['State']=='IL')|(df3['State']=='IN')|(df3['State']=='IA')\
|(df3['State']=='KS')|(df3['State']=='KY')|(df3['State']=='LA')\
|(df3['State']=='ME')|(df3['State']=='MD')|(df3['State']=='MA')\
|(df3['State']=='MI')|(df3['State']=='MN')|(df3['State']=='MS')\
|(df3['State']=='MO')|(df3['State']=='MT')|(df3['State']=='NE')\
|(df3['State']=='NV')|(df3['State']=='NH')|(df3['State']=='NJ')\
|(df3['State']=='NM')|(df3['State']=='NY')|(df3['State']=='NC')\
|(df3['State']=='ND')|(df3['State']=='OH')|(df3['State']=='OK')\
|(df3['State']=='OR')|(df3['State']=='PA')|(df3['State']=='RI')\
|(df3['State']=='SC')|(df3['State']=='SD')|(df3['State']=='TN')\
|(df3['State']=='TX')|(df3['State']=='UT')|(df3['State']=='VT')\
|(df3['State']=='VA')|(df3['State']=='WA')|(df3['State']=='WV')\
|(df3['State']=='WI')|(df3['State']=='WY')]
aliens_df

,Date / Time,City,State,Shape,Duration,Summary,Posted
0,2/24/20 12:45,Chester,NH,Disk,~1 minute,"12:45 PM-shiny, disk/saucer shape seen in dayl...",2/25/20
1,2/24/20 06:00,Medford,OR,Formation,5 minutes,"20 lights in formation across sky. ((""Starlink...",2/25/20
2,2/24/20 05:45,Medford,OR,Other,10 minutes,AT 545 AM I LOOKED UP AND SAID WHAT THE HELL I...,2/25/20
3,2/24/20 05:30,Charlestown,RI,Light,2-3 minutes,light moving at a high rate of speed,2/25/20
4,2/24/20 05:21,Edison,NJ,Light,25 seconds,DURING ISS PASS VIEWING SMALL OBJECT APPEARS T...,2/25/20
...,...,...,...,...,...,...,...
6582,3/1/19 19:12,East Wenatchee,WA,Light,10 seconds,Orange pulsing light.,3/14/19
6583,3/1/19 18:07,Mebane,NC,Changing,Months (since December) l,Interdimensional Beings Caught using thermal v...,3/14/19
6584,3/1/19 13:28,Eagle River,AK,Cylinder,5,Driving down eagle river road looked into sky ...,3/14/19
6585,3/1/19 13:00,Greenville,SC,NaN,10 minutes,Reptilian or Grey alien contact.,8/23/19


In [49]:
# add type column to df
aliens_df_sighting = aliens_df.copy()

aliens_df_sighting['Type']='Sighting'
aliens_df_sighting.head()

,Date / Time,City,State,Shape,Duration,Summary,Posted,Type
0,2/24/20 12:45,Chester,NH,Disk,~1 minute,"12:45 PM-shiny, disk/saucer shape seen in dayl...",2/25/20,Sighting
1,2/24/20 06:00,Medford,OR,Formation,5 minutes,"20 lights in formation across sky. ((""Starlink...",2/25/20,Sighting
2,2/24/20 05:45,Medford,OR,Other,10 minutes,AT 545 AM I LOOKED UP AND SAID WHAT THE HELL I...,2/25/20,Sighting
3,2/24/20 05:30,Charlestown,RI,Light,2-3 minutes,light moving at a high rate of speed,2/25/20,Sighting
4,2/24/20 05:21,Edison,NJ,Light,25 seconds,DURING ISS PASS VIEWING SMALL OBJECT APPEARS T...,2/25/20,Sighting


In [50]:
# adding text column for mapping
aliens_df_sighting['text'] = aliens_df_sighting['Type'].astype(str) + ' : ' + aliens_df_sighting['City'].astype(str)

aliens_df_sighting.head()

,Date / Time,City,State,Shape,Duration,Summary,Posted,Type,text
0,2/24/20 12:45,Chester,NH,Disk,~1 minute,"12:45 PM-shiny, disk/saucer shape seen in dayl...",2/25/20,Sighting,Sighting : Chester
1,2/24/20 06:00,Medford,OR,Formation,5 minutes,"20 lights in formation across sky. ((""Starlink...",2/25/20,Sighting,Sighting : Medford
2,2/24/20 05:45,Medford,OR,Other,10 minutes,AT 545 AM I LOOKED UP AND SAID WHAT THE HELL I...,2/25/20,Sighting,Sighting : Medford
3,2/24/20 05:30,Charlestown,RI,Light,2-3 minutes,light moving at a high rate of speed,2/25/20,Sighting,Sighting : Charlestown
4,2/24/20 05:21,Edison,NJ,Light,25 seconds,DURING ISS PASS VIEWING SMALL OBJECT APPEARS T...,2/25/20,Sighting,Sighting : Edison


In [51]:
aliens_df_sighting.to_csv("Resources/clean_aliens.csv", index = False)

## Retrieve Lat/Long

In [52]:
# Import aliens file as DataFrame
aliens_pd = pd.read_csv("Resources/clean_aliens.csv")
aliens_pd.head()

,Date / Time,City,State,Shape,Duration,Summary,Posted,Type,text
0,2/24/20 12:45,Chester,NH,Disk,~1 minute,"12:45 PM-shiny, disk/saucer shape seen in dayl...",2/25/20,Sighting,Sighting : Chester
1,2/24/20 06:00,Medford,OR,Formation,5 minutes,"20 lights in formation across sky. ((""Starlink...",2/25/20,Sighting,Sighting : Medford
2,2/24/20 05:45,Medford,OR,Other,10 minutes,AT 545 AM I LOOKED UP AND SAID WHAT THE HELL I...,2/25/20,Sighting,Sighting : Medford
3,2/24/20 05:30,Charlestown,RI,Light,2-3 minutes,light moving at a high rate of speed,2/25/20,Sighting,Sighting : Charlestown
4,2/24/20 05:21,Edison,NJ,Light,25 seconds,DURING ISS PASS VIEWING SMALL OBJECT APPEARS T...,2/25/20,Sighting,Sighting : Edison


In [53]:
# Add columns for lat, lng, airport name, airport address, airport rating
# Note that we used "" to specify initial entry.
aliens_pd["Lat"] = ""
aliens_pd["Lng"] = ""
aliens_pd.head()

,Date / Time,City,State,Shape,Duration,Summary,Posted,Type,text,Lat,Lng
0,2/24/20 12:45,Chester,NH,Disk,~1 minute,"12:45 PM-shiny, disk/saucer shape seen in dayl...",2/25/20,Sighting,Sighting : Chester,,
1,2/24/20 06:00,Medford,OR,Formation,5 minutes,"20 lights in formation across sky. ((""Starlink...",2/25/20,Sighting,Sighting : Medford,,
2,2/24/20 05:45,Medford,OR,Other,10 minutes,AT 545 AM I LOOKED UP AND SAID WHAT THE HELL I...,2/25/20,Sighting,Sighting : Medford,,
3,2/24/20 05:30,Charlestown,RI,Light,2-3 minutes,light moving at a high rate of speed,2/25/20,Sighting,Sighting : Charlestown,,
4,2/24/20 05:21,Edison,NJ,Light,25 seconds,DURING ISS PASS VIEWING SMALL OBJECT APPEARS T...,2/25/20,Sighting,Sighting : Edison,,


In [54]:
# Make a copy of aliens_pd
aliens_pd_mod = aliens_pd.copy()

# create a params dict that will be updated with new city each iteration
params = {"key": gkey}

# Loop through the cities_pd and run a lat/long search for each city
# set lists for the dataframe
city_name = []


counter = 0
set_one = 1

for index, row in aliens_pd_mod.iterrows():
    try:
        base_url = "https://maps.googleapis.com/maps/api/geocode/json"

        city = row['City']
        state = row['State']

        # update address key value
        params['address'] = f"{city},{state}"

        # make request
        cities_lat_lng = requests.get(base_url, params=params)

        # print the cities_lat_lng url, avoid doing for public github repos in order to avoid exposing key
        #print(cities_lat_lng.url)

        # convert to json
        cities_lat_lng = cities_lat_lng.json()

        aliens_pd_mod.loc[index, "Lat"] = cities_lat_lng["results"][0]["geometry"]["location"]["lat"]
        aliens_pd_mod.loc[index, "Lng"] = cities_lat_lng["results"][0]["geometry"]["location"]["lng"]
            
        if counter > 99:
            counter = 1
            set_one += 1
            city_name.append(city)
                
        else:
            counter += 1
            city_name.append(city)
        print(f"Processing Record {counter} of Set {set_one} | {city}")
        
    except IndexError:
        print("City not found. Skipping...")
        
    except IndentationError:
        print("City not found. Skipping...")
print("------------------------------\nData Retrieval Complete\n------------------------------")

Processing Record 1 of Set 1 | Chester
Processing Record 2 of Set 1 | Medford
Processing Record 3 of Set 1 | Medford
Processing Record 4 of Set 1 | Charlestown
Processing Record 5 of Set 1 | Edison
Processing Record 6 of Set 1 | North Pole
Processing Record 7 of Set 1 | Stockton
Processing Record 8 of Set 1 | Honolulu
Processing Record 9 of Set 1 | Ambler
Processing Record 10 of Set 1 | Windsor
Processing Record 11 of Set 1 | Greenville
Processing Record 12 of Set 1 | Lake Elmo
Processing Record 13 of Set 1 | Lake Worth Beach
Processing Record 14 of Set 1 | Upper Makefield (Above Delaware River)
Processing Record 15 of Set 1 | Girard
Processing Record 16 of Set 1 | Denver
Processing Record 17 of Set 1 | Newport
Processing Record 18 of Set 1 | Williston
Processing Record 19 of Set 1 | North Kingstown
Processing Record 20 of Set 1 | Denver
Processing Record 21 of Set 1 | Antelope
Processing Record 22 of Set 1 | Selma
Processing Record 23 of Set 1 | Yankton
Processing Record 24 of Set 1 |

Processing Record 95 of Set 2 | Alexandria (near)
Processing Record 96 of Set 2 | Scarborough
Processing Record 97 of Set 2 | Honey Brook
Processing Record 98 of Set 2 | Homer
Processing Record 99 of Set 2 | Atco
Processing Record 100 of Set 2 | Somerton
Processing Record 1 of Set 3 | Orem
Processing Record 2 of Set 3 | Driggs
Processing Record 3 of Set 3 | Mountain Home/Glenns Ferry (between)
Processing Record 4 of Set 3 | Ellenton
Processing Record 5 of Set 3 | San Diego
Processing Record 6 of Set 3 | Lakewood
Processing Record 7 of Set 3 | Telford (UK/England)
Processing Record 8 of Set 3 | Dexter
Processing Record 9 of Set 3 | Groveland
Processing Record 10 of Set 3 | St. Marys
Processing Record 11 of Set 3 | Tomah
Processing Record 12 of Set 3 | Janesville
Processing Record 13 of Set 3 | San Francisco
Processing Record 14 of Set 3 | Red Granite
Processing Record 15 of Set 3 | Boise
Processing Record 16 of Set 3 | Snow Hill
Processing Record 17 of Set 3 | Puyallup
Processing Record

Processing Record 90 of Set 4 | Warren
Processing Record 91 of Set 4 | Colonial Heights
Processing Record 92 of Set 4 | Varnville
Processing Record 93 of Set 4 | Palm Harbor
Processing Record 94 of Set 4 | Longs
Processing Record 95 of Set 4 | North Hollywood
Processing Record 96 of Set 4 | Davis
Processing Record 97 of Set 4 | Natchitoches
Processing Record 98 of Set 4 | Deming
Processing Record 99 of Set 4 | Shoreline
Processing Record 100 of Set 4 | Honolulu
Processing Record 1 of Set 5 | Woodland
Processing Record 2 of Set 5 | N. Myrtle Beach
Processing Record 3 of Set 5 | Las Vegas
Processing Record 4 of Set 5 | Asheville
Processing Record 5 of Set 5 | North Port
Processing Record 6 of Set 5 | Valley City
Processing Record 7 of Set 5 | Green River
Processing Record 8 of Set 5 | Royal Palm Beach
Processing Record 9 of Set 5 | Grand Island
Processing Record 10 of Set 5 | Fishersville
Processing Record 11 of Set 5 | Bangor
Processing Record 12 of Set 5 | Lakewood
Processing Record 13

Processing Record 84 of Set 6 | Many Farms
Processing Record 85 of Set 6 | Albuquerque
Processing Record 86 of Set 6 | Along HWY 550 Near Chaco Canyon
Processing Record 87 of Set 6 | Riverside
Processing Record 88 of Set 6 | Jasper
Processing Record 89 of Set 6 | Jasper
Processing Record 90 of Set 6 | Patterson
Processing Record 91 of Set 6 | Jonesboro
Processing Record 92 of Set 6 | Issaquah
Processing Record 93 of Set 6 | Newburgh
Processing Record 94 of Set 6 | Windsor Heights
Processing Record 95 of Set 6 | Bristolville
Processing Record 96 of Set 6 | Boulder City
Processing Record 97 of Set 6 | Cambridge
Processing Record 98 of Set 6 | Phillipsburg
Processing Record 99 of Set 6 | Danielson
Processing Record 100 of Set 6 | Northbrook
Processing Record 1 of Set 7 | Northbrook
Processing Record 2 of Set 7 | Wheat Ridge
Processing Record 3 of Set 7 | Palm Coast
Processing Record 4 of Set 7 | Morro Bay
Processing Record 5 of Set 7 | Morro Bay
Processing Record 6 of Set 7 | Saint August

Processing Record 76 of Set 8 | Pueblo
Processing Record 77 of Set 8 | Eagan
Processing Record 78 of Set 8 | Jenks
Processing Record 79 of Set 8 | Dallas
Processing Record 80 of Set 8 | Austin
Processing Record 81 of Set 8 | Corpus Christi
Processing Record 82 of Set 8 | Dallas
Processing Record 83 of Set 8 | Dracut
Processing Record 84 of Set 8 | Jonesboro
Processing Record 85 of Set 8 | Lincoln
Processing Record 86 of Set 8 | Springfield
Processing Record 87 of Set 8 | Lubbock
Processing Record 88 of Set 8 | New Bern
Processing Record 89 of Set 8 | North Berwick
Processing Record 90 of Set 8 | Hellertown
Processing Record 91 of Set 8 | Greensboro
Processing Record 92 of Set 8 | Lincoln
Processing Record 93 of Set 8 | Shreveport
Processing Record 94 of Set 8 | Kingston
Processing Record 95 of Set 8 | Shreveport
Processing Record 96 of Set 8 | Bear
Processing Record 97 of Set 8 | Charleston
Processing Record 98 of Set 8 | West Odessa
Processing Record 99 of Set 8 | Buckeye
Processing R

...

Processing Record 38 of Set 54 | New Orleans
Processing Record 39 of Set 54 | Concord
Processing Record 40 of Set 54 | Rock Creek
Processing Record 41 of Set 54 | Springfield
Processing Record 42 of Set 54 | Aurora
Processing Record 43 of Set 54 | Garrett
Processing Record 44 of Set 54 | Zionsville
Processing Record 45 of Set 54 | Olive Branch
Processing Record 46 of Set 54 | Goffstown
Processing Record 47 of Set 54 | Burlington/Billerica
Processing Record 48 of Set 54 | Seatac
Processing Record 49 of Set 54 | Chicago
Processing Record 50 of Set 54 | Mill Creek
Processing Record 51 of Set 54 | Vallejo
Processing Record 52 of Set 54 | Austin
Processing Record 53 of Set 54 | Old Town
Processing Record 54 of Set 54 | Goffstown
Processing Record 55 of Set 54 | Shawnee
Processing Record 56 of Set 54 | Lake Charles
Processing Record 57 of Set 54 | Columbus
Processing Record 58 of Set 54 | Baton Rouge
Processing Record 59 of Set 54 | Burnsville
Processing Record 60 of Set 54 | Milledgeville
P

Processing Record 28 of Set 56 | New York City (Queens)
Processing Record 29 of Set 56 | Punta Gorda
Processing Record 30 of Set 56 | Fullerton
Processing Record 31 of Set 56 | Mesa
Processing Record 32 of Set 56 | Eagle Mountain
Processing Record 33 of Set 56 | Eagle Mountain
Processing Record 34 of Set 56 | Chesapeake
Processing Record 35 of Set 56 | Thompson's Station
Processing Record 36 of Set 56 | Indian Lake Estates
Processing Record 37 of Set 56 | Norwalk
Processing Record 38 of Set 56 | Phoenix
Processing Record 39 of Set 56 | Strongsville
Processing Record 40 of Set 56 | Nappanee
Processing Record 41 of Set 56 | Alexandria
Processing Record 42 of Set 56 | Akron
Processing Record 43 of Set 56 | St. Petersburg
Processing Record 44 of Set 56 | Van Nuys
Processing Record 45 of Set 56 | Battle Creek
Processing Record 46 of Set 56 | Lexington
Processing Record 47 of Set 56 | North Wilkesboro
Processing Record 48 of Set 56 | Kentwood
Processing Record 49 of Set 56 | Dauphin Island
P

Processing Record 17 of Set 58 | Espanola
Processing Record 18 of Set 58 | Santa Fe
Processing Record 19 of Set 58 | Boise
Processing Record 20 of Set 58 | Sedona
Processing Record 21 of Set 58 | Lansing
Processing Record 22 of Set 58 | Park Township (Holland)
Processing Record 23 of Set 58 | Fairfax
Processing Record 24 of Set 58 | Baker
Processing Record 25 of Set 58 | Palmdale
Processing Record 26 of Set 58 | Brookings
Processing Record 27 of Set 58 | Orlando
Processing Record 28 of Set 58 | Nashville
Processing Record 29 of Set 58 | Lee's Summit
Processing Record 30 of Set 58 | Minneapolis
Processing Record 31 of Set 58 | Las Vegas
Processing Record 32 of Set 58 | Harrisburg
Processing Record 33 of Set 58 | Mary Esther
Processing Record 34 of Set 58 | Goffstown
Processing Record 35 of Set 58 | Alexandria
Processing Record 36 of Set 58 | Alexandria
Processing Record 37 of Set 58 | California City
Processing Record 38 of Set 58 | Huson
Processing Record 39 of Set 58 | Fairview
Proces

Processing Record 7 of Set 60 | Kansas City
Processing Record 8 of Set 60 | Columbus
Processing Record 9 of Set 60 | Tucson
Processing Record 10 of Set 60 | Baltimore
Processing Record 11 of Set 60 | Windsor
Processing Record 12 of Set 60 | Tucson
Processing Record 13 of Set 60 | Reading
Processing Record 14 of Set 60 | Morris
Processing Record 15 of Set 60 | South Weymouth
Processing Record 16 of Set 60 | Palm Coast
Processing Record 17 of Set 60 | Jacksonville
Processing Record 18 of Set 60 | Jacksonville
Processing Record 19 of Set 60 | Taos
Processing Record 20 of Set 60 | Sheboygan
Processing Record 21 of Set 60 | Rocky Hill
Processing Record 22 of Set 60 | Meldrim
Processing Record 23 of Set 60 | Elizabeth
Processing Record 24 of Set 60 | Windsor
Processing Record 25 of Set 60 | Memphis
Processing Record 26 of Set 60 | Nampa
Processing Record 27 of Set 60 | North Dartmouth
Processing Record 28 of Set 60 | Turners Falls
Processing Record 29 of Set 60 | Wright
Processing Record 30 

Processing Record 95 of Set 61 | Fairmount
Processing Record 96 of Set 61 | Nipomo
Processing Record 97 of Set 61 | Lake Como
Processing Record 98 of Set 61 | Peoria
Processing Record 99 of Set 61 | Polacca
Processing Record 100 of Set 61 | Polacca
Processing Record 1 of Set 62 | Pittsburgh
Processing Record 2 of Set 62 | New River
Processing Record 3 of Set 62 | Marshall
Processing Record 4 of Set 62 | Mount Vernon
Processing Record 5 of Set 62 | Nampa
Processing Record 6 of Set 62 | Gainesville
Processing Record 7 of Set 62 | Unincorporated Miami-Dade County
Processing Record 8 of Set 62 | Voorheesville
Processing Record 9 of Set 62 | Ypsilanti
Processing Record 10 of Set 62 | East Wenatchee
Processing Record 11 of Set 62 | Mebane
Processing Record 12 of Set 62 | Eagle River
Processing Record 13 of Set 62 | Greenville
Processing Record 14 of Set 62 | Red Wing
------------------------------
Data Retrieval Complete
------------------------------


In [55]:
# Visualize to confirm lat lng appear
aliens_pd_mod.head()

,Date / Time,City,State,Shape,Duration,Summary,Posted,Type,text,Lat,Lng
0,2/24/20 12:45,Chester,NH,Disk,~1 minute,"12:45 PM-shiny, disk/saucer shape seen in dayl...",2/25/20,Sighting,Sighting : Chester,42.9568,-71.2573
1,2/24/20 06:00,Medford,OR,Formation,5 minutes,"20 lights in formation across sky. ((""Starlink...",2/25/20,Sighting,Sighting : Medford,42.3265,-122.876
2,2/24/20 05:45,Medford,OR,Other,10 minutes,AT 545 AM I LOOKED UP AND SAID WHAT THE HELL I...,2/25/20,Sighting,Sighting : Medford,42.3265,-122.876
3,2/24/20 05:30,Charlestown,RI,Light,2-3 minutes,light moving at a high rate of speed,2/25/20,Sighting,Sighting : Charlestown,41.3832,-71.6419
4,2/24/20 05:21,Edison,NJ,Light,25 seconds,DURING ISS PASS VIEWING SMALL OBJECT APPEARS T...,2/25/20,Sighting,Sighting : Edison,40.5187,-74.4121


In [56]:
aliens_pd_mod.to_csv("Resources/clean_aliens_lat_long.csv", index = False)

In [57]:
# Import aliens file as DataFrame
aliens_pd_mod = pd.read_csv("Resources/clean_aliens_lat_long.csv")
aliens_pd_mod.head()

,Date / Time,City,State,Shape,Duration,Summary,Posted,Type,text,Lat,Lng
0,2/24/20 12:45,Chester,NH,Disk,~1 minute,"12:45 PM-shiny, disk/saucer shape seen in dayl...",2/25/20,Sighting,Sighting : Chester,42.956753,-71.257285
1,2/24/20 06:00,Medford,OR,Formation,5 minutes,"20 lights in formation across sky. ((""Starlink...",2/25/20,Sighting,Sighting : Medford,42.326515,-122.875595
2,2/24/20 05:45,Medford,OR,Other,10 minutes,AT 545 AM I LOOKED UP AND SAID WHAT THE HELL I...,2/25/20,Sighting,Sighting : Medford,42.326515,-122.875595
3,2/24/20 05:30,Charlestown,RI,Light,2-3 minutes,light moving at a high rate of speed,2/25/20,Sighting,Sighting : Charlestown,41.383216,-71.641874
4,2/24/20 05:21,Edison,NJ,Light,25 seconds,DURING ISS PASS VIEWING SMALL OBJECT APPEARS T...,2/25/20,Sighting,Sighting : Edison,40.518715,-74.412095


In [107]:
sightings = aliens_pd_mod.copy()

In [108]:
# Dropping old Location and Refs columns 
sightings.drop(columns =["Date / Time","State","Shape","Duration","Summary","Posted"], inplace = True) 

In [109]:
sightings.rename(columns={"City": "Site Name", "Lng": "Long"}, inplace=True)

In [128]:
sightings['Color'] = '#00FE35'

In [130]:
sightings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6121 entries, 0 to 6120
Data columns (total 6 columns):
Site Name    6101 non-null object
Type         6121 non-null object
text         6121 non-null object
Lat          6114 non-null float64
Long         6114 non-null float64
Color        6121 non-null object
dtypes: float64(2), object(4)
memory usage: 287.0+ KB


## Scrape Nuclear Sites

In [58]:
# URL for the source of the data on Wikipedia
url = 'https://en.wikipedia.org/wiki/List_of_nuclear_power_stations'

In [59]:
# Use pandas to get the tables from the Wikipedia page and create a list
# of dictionaries
tables = pd.read_html(url,encoding='utf-8')
tables

[                                          0
 0  Map all coordinates using: OpenStreetMap
 1        Download coordinates as: KML · GPX,
     Power station # units[note 1] Net capacity[note 2](MWe)        Country  \
 0         Almaraz               2                      2017          Spain   
 1           Angra               2             1,884[note 3]         Brazil   
 2             ANO               2                      1839  United States   
 3            Ascó               2                      1992          Spain   
 4          Atucha               2                      1027      Argentina   
 ..            ...             ...                       ...            ...   
 146     Watts Bar               2                      2288  United States   
 147    Wolf Creek               1                      1160  United States   
 148      Wolseong      5[note 45]                      3835    South Korea   
 149     Yangjiang               6                      6000          Chin

In [60]:
# Isolate the second table from the list of dictionaries and make that dict a dataframe
df = tables[1]

In [61]:
# View the column names
df.columns

Index(['Power station', '# units[note 1]', 'Net capacity[note 2](MWe)',
       'Country', 'Location', 'Refs'],
      dtype='object')

In [62]:
# Change the column names
df.columns = ['Station', 'Units', 'Capacity_MWe', 'Country', 
              'Location', 'Refs']

In [63]:
# Both DMS and Geopoint Lat and Long data is in one column 
# we are only interested in the Geopoint Lat/Long 
coordinates = df["Location"].str.split(" / ", n = 1, expand = True)

In [64]:
# Create a new column in the dataframe
df["GP_Lat_Long"]= coordinates[1]

In [65]:
# Dropping old Location and Refs columns 
df.drop(columns =["Location","Refs"], inplace = True) 

In [66]:
# Split the column of data GP_Lat_long that contains two coordinates into 
# two separate columns
DMS_coordinates = df["GP_Lat_Long"].str.split(" ", n = 1, expand = True)
df["GP_Lat"]= DMS_coordinates[0] 
df["GP_Long"]= DMS_coordinates[1]

In [67]:
# Create a function to transform the Latitude and Longitude points from the format
# 41.2000°S to -41.2000 for example 
def dms2dd(s):
    # example: s = """0°51'56.29"S"""
    coord, direction = re.split('°', s)
    coord = coord.replace(u'\ufeff','')
    coord = float(coord)
    if direction in ('S','W'):
        coord*= -1
    return coord

In [68]:
# Transform the Latitude and Longitude values to be either negative or positive
# Values for South and West are negative and North and East are positive
df['Latitude'] = df['GP_Lat'].apply(dms2dd)
df['Longitude'] = df['GP_Long'].apply(dms2dd)

In [69]:
# Remove the columns that are no longer necessary
df.drop(columns =["GP_Lat_Long","GP_Lat","GP_Long"], inplace = True)  

In [70]:
# Create a United States only slice of the dataframe
us = df[df["Country"] == "United States"]

In [71]:
# Add column "Type" with the type of point name
us_copy = us.copy()

us['Type'] = 'Nuclear Site'
us['Color'] = '#ff0000'
# Add column "text" with a descrition of the point for plottin in plotly
us['text'] = us['Type'].astype(str) + ' : ' + us['Station'].astype(str)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/panda

In [72]:
us.drop(columns =["Units","Capacity_MWe","Country"], inplace = True) 

In [73]:
us.rename(columns={"Station": "Site Name","Latitude": "Lat", "Longitude": "Long"}, inplace=True)

In [74]:
us.head()

,Site Name,Lat,Long,Type,Color,text
2,ANO,35.31028,-93.23139,Nuclear Site,#ff0000,Nuclear Site : ANO
6,Beaver Valley,40.62333,-80.43056,Nuclear Site,#ff0000,Nuclear Site : Beaver Valley
10,Braidwood,41.24361,-88.22917,Nuclear Site,#ff0000,Nuclear Site : Braidwood
12,Browns Ferry,34.70389,-87.11861,Nuclear Site,#ff0000,Nuclear Site : Browns Ferry
14,Brunswick,33.95833,-78.01028,Nuclear Site,#ff0000,Nuclear Site : Brunswick


In [75]:
us.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 46 entries, 2 to 147
Data columns (total 6 columns):
Site Name    46 non-null object
Lat          46 non-null float64
Long         46 non-null float64
Type         46 non-null object
Color        46 non-null object
text         46 non-null object
dtypes: float64(2), object(4)
memory usage: 2.5+ KB


In [76]:
us.to_csv("Resources/scraped_wiki_nuclear_sites.csv", index = False)

### Military Bases

In [77]:
# import excel file for military bases
military_base_df = pd.read_excel (r'Resources/military-bases.xlsx')
military_base_df = military_base_df.dropna()

military_base_df.head()

,Geo Point,OBJECTID,COMPONENT,Site Name,Joint Base,State Terr,COUNTRY,Oper Stat,Shape St Are,Shape St Len
13,"38.805160992, -76.8744374247",144,AF Active,Andrews AFB,Joint Base Andrews - Naval Air Facility Washin...,Maryland,United States,Active,2.938397e+07,22642.684174
15,"32.9631678645, -79.9639295206",113,AF Active,NWS Charleston,Joint Base Charleston,South Carolina,United States,Active,9.684911e+07,160700.657287
28,"13.5179166878, 144.811096965",239,Navy Active,Tumon Tank Farm,Joint Region Marianas,Guam,Guam,Active,1.023036e+05,1749.477641
30,"13.5867134229, 144.862326053",240,Navy Active,Potts Tank Farm,Joint Region Marianas,Guam,Guam,Active,8.480141e+04,1181.131174
32,"38.8786170847, -77.075713083",718,Army Active,Fort Myer,Joint Base Myer - Henderson Hall,Virginia,United States,Active,2.065523e+06,12833.022256


In [78]:
# drop unnecessary columns
military_base_df.drop(columns= ['Shape St Are','Shape St Len'], inplace=True)

In [79]:
# reset index
military_base_df.set_index('Site Name')
military_base_df.head()

,Geo Point,OBJECTID,COMPONENT,Site Name,Joint Base,State Terr,COUNTRY,Oper Stat
13,"38.805160992, -76.8744374247",144,AF Active,Andrews AFB,Joint Base Andrews - Naval Air Facility Washin...,Maryland,United States,Active
15,"32.9631678645, -79.9639295206",113,AF Active,NWS Charleston,Joint Base Charleston,South Carolina,United States,Active
28,"13.5179166878, 144.811096965",239,Navy Active,Tumon Tank Farm,Joint Region Marianas,Guam,Guam,Active
30,"13.5867134229, 144.862326053",240,Navy Active,Potts Tank Farm,Joint Region Marianas,Guam,Guam,Active
32,"38.8786170847, -77.075713083",718,Army Active,Fort Myer,Joint Base Myer - Henderson Hall,Virginia,United States,Active


In [80]:
# format the lat and long to two columns
split_data = military_base_df["Geo Point"].str.split(",", n=1, expand=True)

military_base_df['Lat'] = split_data[0]
military_base_df['Long'] = split_data[1]

military_base_df.head()

,Geo Point,OBJECTID,COMPONENT,Site Name,Joint Base,State Terr,COUNTRY,Oper Stat,Lat,Long
13,"38.805160992, -76.8744374247",144,AF Active,Andrews AFB,Joint Base Andrews - Naval Air Facility Washin...,Maryland,United States,Active,38.805160992,-76.8744374247
15,"32.9631678645, -79.9639295206",113,AF Active,NWS Charleston,Joint Base Charleston,South Carolina,United States,Active,32.9631678645,-79.9639295206
28,"13.5179166878, 144.811096965",239,Navy Active,Tumon Tank Farm,Joint Region Marianas,Guam,Guam,Active,13.5179166878,144.811096965
30,"13.5867134229, 144.862326053",240,Navy Active,Potts Tank Farm,Joint Region Marianas,Guam,Guam,Active,13.5867134229,144.862326053
32,"38.8786170847, -77.075713083",718,Army Active,Fort Myer,Joint Base Myer - Henderson Hall,Virginia,United States,Active,38.8786170847,-77.075713083


In [81]:
# add type column to df
military_base_df['Type']='Military Base'
military_base_df.head()

,Geo Point,OBJECTID,COMPONENT,Site Name,Joint Base,State Terr,COUNTRY,Oper Stat,Lat,Long,Type
13,"38.805160992, -76.8744374247",144,AF Active,Andrews AFB,Joint Base Andrews - Naval Air Facility Washin...,Maryland,United States,Active,38.805160992,-76.8744374247,Military Base
15,"32.9631678645, -79.9639295206",113,AF Active,NWS Charleston,Joint Base Charleston,South Carolina,United States,Active,32.9631678645,-79.9639295206,Military Base
28,"13.5179166878, 144.811096965",239,Navy Active,Tumon Tank Farm,Joint Region Marianas,Guam,Guam,Active,13.5179166878,144.811096965,Military Base
30,"13.5867134229, 144.862326053",240,Navy Active,Potts Tank Farm,Joint Region Marianas,Guam,Guam,Active,13.5867134229,144.862326053,Military Base
32,"38.8786170847, -77.075713083",718,Army Active,Fort Myer,Joint Base Myer - Henderson Hall,Virginia,United States,Active,38.8786170847,-77.075713083,Military Base


In [82]:
# adding text column for mapping
military_base_df['text'] = military_base_df['Type'].astype(str) + ' : ' + military_base_df['Site Name'].astype(str)

military_base_df.head()

,Geo Point,OBJECTID,COMPONENT,Site Name,Joint Base,State Terr,COUNTRY,Oper Stat,Lat,Long,Type,text
13,"38.805160992, -76.8744374247",144,AF Active,Andrews AFB,Joint Base Andrews - Naval Air Facility Washin...,Maryland,United States,Active,38.805160992,-76.8744374247,Military Base,Military Base : Andrews AFB
15,"32.9631678645, -79.9639295206",113,AF Active,NWS Charleston,Joint Base Charleston,South Carolina,United States,Active,32.9631678645,-79.9639295206,Military Base,Military Base : NWS Charleston
28,"13.5179166878, 144.811096965",239,Navy Active,Tumon Tank Farm,Joint Region Marianas,Guam,Guam,Active,13.5179166878,144.811096965,Military Base,Military Base : Tumon Tank Farm
30,"13.5867134229, 144.862326053",240,Navy Active,Potts Tank Farm,Joint Region Marianas,Guam,Guam,Active,13.5867134229,144.862326053,Military Base,Military Base : Potts Tank Farm
32,"38.8786170847, -77.075713083",718,Army Active,Fort Myer,Joint Base Myer - Henderson Hall,Virginia,United States,Active,38.8786170847,-77.075713083,Military Base,Military Base : Fort Myer


In [83]:
military_base_df.drop(columns =["Geo Point","OBJECTID","COMPONENT","Joint Base","State Terr","COUNTRY","Oper Stat"], inplace = True) 

In [84]:
military_base_df['Color'] = '#636EFA'

In [85]:
military_base_df['Lat'] = military_base_df['Lat'].astype(float)
military_base_df['Long'] = military_base_df['Long'].astype(float)

In [86]:
military_base_df.head()

,Site Name,Lat,Long,Type,text,Color
13,Andrews AFB,38.805161,-76.874437,Military Base,Military Base : Andrews AFB,#636EFA
15,NWS Charleston,32.963168,-79.963930,Military Base,Military Base : NWS Charleston,#636EFA
28,Tumon Tank Farm,13.517917,144.811097,Military Base,Military Base : Tumon Tank Farm,#636EFA
30,Potts Tank Farm,13.586713,144.862326,Military Base,Military Base : Potts Tank Farm,#636EFA
32,Fort Myer,38.878617,-77.075713,Military Base,Military Base : Fort Myer,#636EFA


In [87]:
military_base_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 62 entries, 13 to 691
Data columns (total 6 columns):
Site Name    62 non-null object
Lat          62 non-null float64
Long         62 non-null float64
Type         62 non-null object
text         62 non-null object
Color        62 non-null object
dtypes: float64(2), object(4)
memory usage: 3.4+ KB


### Combine nuclear and military base sites

In [131]:
dfs_list = [us,military_base_df,sightings]
sites = pd.concat(dfs_list, sort=False)
sites.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6229 entries, 2 to 6120
Data columns (total 6 columns):
Site Name    6209 non-null object
Lat          6222 non-null float64
Long         6222 non-null float64
Type         6229 non-null object
Color        6229 non-null object
text         6229 non-null object
dtypes: float64(2), object(4)
memory usage: 340.6+ KB


### Map of Military bases and Nuclear sites

In [133]:
fig = go.Figure(data=go.Scattergeo(
        lon = sites['Long'],
        lat = sites['Lat'],
        text = sites['text'],
        mode = 'markers',
        marker_color = sites['Color'],
        #size = us['Capacity_MWe']
        ))

fig.update_layout(
        title = 'Nuclear (red) /Military (blue) Facilities with UFO sightings (green)<br>(Hover for facility names)',
        geo_scope='usa',
    )
fig.show()

## DB

In [90]:
#connect to a local db
rds_connection_string = "postgres:postgres@localhost:5432/aliens"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [91]:
engine.table_names()

['aliens_sighting', 'us_nuclear_sites']

In [92]:
#aliens_pd.to_sql(name='aliens_sighting', con=engine, if_exists='append', index=False)

In [93]:
us.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 46 entries, 2 to 147
Data columns (total 6 columns):
Site Name    46 non-null object
Lat          46 non-null float64
Long         46 non-null float64
Type         46 non-null object
Color        46 non-null object
text         46 non-null object
dtypes: float64(2), object(4)
memory usage: 2.5+ KB


In [94]:
us.to_sql(name='us_nuclear_sites', con=engine, if_exists='append', index=False)

ProgrammingError: (psycopg2.errors.UndefinedColumn) column "Site Name" of relation "us_nuclear_sites" does not exist
LINE 1: INSERT INTO us_nuclear_sites ("Site Name", "Lat", "Long", "T...
                                      ^

[SQL: INSERT INTO us_nuclear_sites ("Site Name", "Lat", "Long", "Type", "Color", text) VALUES (%(Site Name)s, %(Lat)s, %(Long)s, %(Type)s, %(Color)s, %(text)s)]
[parameters: ({'Site Name': 'ANO', 'Lat': 35.31028, 'Long': -93.23139, 'Type': 'Nuclear Site', 'Color': '#ff0000', 'text': 'Nuclear Site : ANO'}, {'Site Name': 'Beaver Valley', 'Lat': 40.62333, 'Long': -80.43056, 'Type': 'Nuclear Site', 'Color': '#ff0000', 'text': 'Nuclear Site : Beaver Valley'}, {'Site Name': 'Braidwood', 'Lat': 41.24361, 'Long': -88.22917, 'Type': 'Nuclear Site', 'Color': '#ff0000', 'text': 'Nuclear Site : Braidwood'}, {'Site Name': 'Browns Ferry', 'Lat': 34.70389, 'Long': -87.11861, 'Type': 'Nuclear Site', 'Color': '#ff0000', 'text': 'Nuclear Site : Browns Ferry'}, {'Site Name': 'Br

In [95]:
military_base_df.to_sql(name='military_bases', con=engine, if_exists='append', index=False)

In [96]:
pd.read_sql_query('select * from aliens_sighting', con=engine).head()

,Date / Time,City,State,Shape,Duration,Summary,Posted,Lat,Lng
0,2/24/20 12:45,Chester,NH,Disk,~1 minute,"12:45 PM-shiny, disk/saucer shape seen in dayl...",2/25/20,,
1,2/24/20 06:00,Medford,OR,Formation,5 minutes,"20 lights in formation across sky. ((""Starlink...",2/25/20,,
2,2/24/20 05:45,Medford,OR,Other,10 minutes,AT 545 AM I LOOKED UP AND SAID WHAT THE HELL I...,2/25/20,,
3,2/24/20 05:30,Charlestown,RI,Light,2-3 minutes,light moving at a high rate of speed,2/25/20,,
4,2/24/20 05:21,Edison,NJ,Light,25 seconds,DURING ISS PASS VIEWING SMALL OBJECT APPEARS T...,2/25/20,,


In [97]:
pd.read_sql_query('select * from us_nuclear_sites', con=engine).head()

,Station,Units,Capacity_MWe,Country,Latitude,Longitude
0,ANO,2,1839,United States,35.31028,-93.23139
1,Beaver Valley,2,1738,United States,40.62333,-80.43056
2,Braidwood,2,2330,United States,41.24361,-88.22917
3,Browns Ferry,3,3300,United States,34.70389,-87.11861
4,Brunswick,2,1858,United States,33.95833,-78.01028


In [98]:
pd.read_sql_query('select * from military_bases', con=engine).head()

,Site Name,Lat,Long,Type,text,Color
0,Andrews AFB,38.805161,-76.874437,Military Base,Military Base : Andrews AFB,#636EFA
1,NWS Charleston,32.963168,-79.963930,Military Base,Military Base : NWS Charleston,#636EFA
2,Tumon Tank Farm,13.517917,144.811097,Military Base,Military Base : Tumon Tank Farm,#636EFA
3,Potts Tank Farm,13.586713,144.862326,Military Base,Military Base : Potts Tank Farm,#636EFA
4,Fort Myer,38.878617,-77.075713,Military Base,Military Base : Fort Myer,#636EFA
